In [7]:
import mrigutilities as mu
import pandas as pd
import mrigstatics 
engine = mu.sql_engine()
engine_mrigweb = mu.sql_engine(dbname=mrigstatics.MRIGWEB[mrigstatics.ENVIRONMENT])


In [30]:
stock_date = pd.read_sql("select series as item,max(date) as last_date from stock_history group by series",engine)
fo_date = pd.read_sql("select 'FO' as item,max(date) as last_date from futures_options_history",engine)
mf_date = pd.read_sql("select 'MF' as item ,max(nav_date) as last_date from mf_history",engine)
webpage_date = pd.read_sql("select 'WEB' as item, max(load_date) as last_date from webpages",engine_mrigweb)

In [55]:
df = pd.concat([stock_date, fo_date,mf_date,webpage_date])
df['last_date'] = df['last_date'].apply(lambda x: x.strftime('%d-%m-%Y'))
df = df.set_index('item').transpose()
status = ""
for col in df.columns:
    status = status +  '||'+col+' - '+df[col]
status

last_date    ||EQ - 26-03-2024||IN - 26-03-2024||TRI - 04-0...
dtype: object

In [16]:
industry = pd.read_csv('industry.csv')
industry = industry[['symbol','benchmark']]
industry

,symbol,benchmark
0,20MICRONS,NaN
1,21STCENMGM,NaN
2,3IINFOTECH,NIFTY IT
3,3MINDIA,NaN
4,3PLAND,NaN
...,...,...
1771,ZOTA,NaN
1772,ZUARI,NaN
1773,ZUARIGLOB,NaN
1774,ZYDUSWELL,NaN


In [19]:
sql = '''
drop table if exists temp_scm;

create table temp_scm 
(
  symbol text not null,
  benchmark text
);
'''
engine.execute(sql)

industry.to_sql('temp_scm',engine, if_exists='append', index=False)
sql = '''
update security_master 
set benchmark = ts.benchmark
from temp_scm as ts
where security_master.symbol = ts.symbol
'''
engine.execute(sql)
